In [1]:
import os
import re

from dotenv import load_dotenv
from langchain import hub
from langchain.agents import AgentType, load_tools
from langchain.agents.initialize import initialize_agent
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
# from langchain.chat_models import ChatOpenAI
# from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain.tools import ArxivQueryRun, WikipediaQueryRun
from langchain.tools.render import render_text_description_and_args
from langchain.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain.prompts import MessagesPlaceholder
# from langchain.schema import ChatMessage, SystemMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.base import BaseCallbackManager

from llamp.mp.agents import (
    MPSummaryExpert,
    MPThermoExpert,
    MPElasticityExpert,
    MPDielectricExpert,
    MPPiezoelectricExpert,
    MPMagnetismExpert,
    MPElectronicExpert,
    MPSynthesisExpert,
    MPStructureRetriever,
)
from llamp.arxiv.agents import ArxivAgent

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)
OPENAI_ORGANIZATION = os.getenv("OPENAI_ORGANIZATION", None)

OPENAI_GPT_MODEL = "gpt-4-1106-preview"
# OPENAI_GPT_MODEL = "gpt-3.5-turbo-1106"
# OPENAI_GPT_MODEL = "gpt-4"

No module named 'phonopy'
No module named 'phonopy'


In [2]:
top_llm = ChatOpenAI(
    temperature=0.1,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

bottom_callback_handler = StreamingStdOutCallbackHandler()

bottom_llm = ChatOpenAI(
    temperature=0,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    max_retries=5,
    streaming=True,
    callbacks=[bottom_callback_handler],
)


wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper())

tools = [
    MPThermoExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPElasticityExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPDielectricExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPMagnetismExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPElectronicExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPPiezoelectricExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPSummaryExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPSynthesisExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPStructureRetriever(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    # ArxivAgent(llm=bottom_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    arxiv,
    wikipedia,
]
tools += load_tools(["llm-math"], llm=bottom_llm)

prompt = hub.pull("hwchase17/react-multi-input-json")
prompt.messages[0].prompt.template = (
    re.sub(
        r"\s+",
        " ",
        """You are a data-aware agent that can consult materials-related
    data through Materials Project (MP) database, arXiv, and Wikipedia. Ask 
    user to clarify their queries if needed. Please note that you don't have 
    direct control over MP but through multiple assistant agents to help you. 
    You need to provide complete context in the input for them to do their job.
    """,
    ).replace("\n", " ")
    + prompt.messages[0].prompt.template
)

prompt = prompt.partial(
    tools=render_text_description_and_args(tools),
    tool_names=", ".join([t.name for t in tools]),
)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | top_llm.bind(stop=["Observation"])
    # | map_reduce_chain  # TODO: Add map-reduce after LLM
    | JSONAgentOutputParser()
)

conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True
)

agent_kwargs = {
    "handle_parsing_errors": True,
    "extra_prompt_messages": [
        MessagesPlaceholder(variable_name="chat_history"),
    ],
    "early_stopping_method": "generate",
}

agent_executor = initialize_agent(
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=top_llm,
    verbose=True,
    max_iterations=20,
    memory=conversational_memory,
    agent_kwargs=agent_kwargs,
    handle_parsing_errors=True,
    callback_manager=BaseCallbackManager(handlers=[bottom_callback_handler]),
)

/home/cyrus/miniconda3/envs/llamp/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [3]:
agent_executor.invoke(
    {
        "input": "What is the magnetic ordering of stable FePO4? Give me its material id, space group, and magnetization per formula unit.",
    }
)



> Entering new AgentExecutor chain...
To find the magnetic ordering of stable FePO_4, along with its material ID, space group, and magnetization per formula unit, I will use the MPMagnetismExpert tool to query the Materials Project database.

Action:
```json
{
  "action": "MPMagnetismExpert",
  "action_input": {
    "input": "What is the magnetic ordering of stable FePO4?"
  }
}
```To find the magnetic ordering of stable FePO_4, along with its material ID, space group, and magnetization per formula unit, I will use the MPMagnetismExpert tool to query the Materials Project database.

Action:
```json
{
  "action": "MPMagnetismExpert",
  "action_input": {
    "input": "What is the magnetic ordering of stable FePO4?"
  }
}
```

> Entering new AgentExecutor chain...
Action:
```
{
  "action": "search_materials_magnetism__get",
  "action_input": {
    "formula": "FePO4"
  }
}
```Action:
```
{
  "action": "search_materials_magnetism__get",
  "action_input": {
    "formula": "FePO4"
  }
}
```

Retrieving SummaryDoc documents:   0%|          | 0/30 [00:00<?, ?it/s]

Retrieving MagnetismDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

[{'formula_pretty': 'FePO4', 'material_id': 'mp-19109', 'ordering': 'FM', 'is_magnetic': True, 'exchange_symmetry': 152, 'num_magnetic_sites': 3, 'num_unique_magnetic_sites': 1, 'types_of_magnetic_species': ['Fe'], 'magmoms': [4.1, 4.1, 4.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'total_magnetization': 15.0000006, 'total_magnetization_normalized_vol': 0.06026982889365446, 'total_magnetization_normalized_formula_units': 5.0000002}]Action:
```
{
  "action": "Final Answer",
  "action_input": "The magnetic ordering of stable FePO4 is ferromagnetic (FM)."
}
```Action:
```
{
  "action": "Final Answer",
  "action_input": "The magnetic ordering of stable FePO4 is ferromagnetic (FM)."
}
```

> Finished chain.

Observation: {'input': 'What is the magnetic ordering of stable FePO4?', 'output': 'The magnetic ordering of stable FePO4 is ferromagnetic (FM).'}
Thought:Action:
```json
{
  "action": "MPSummaryExpert",
  "action_input": {
    "input": "Give me the su

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

[{'formula_pretty': 'FePO4', 'symmetry': {'crystal_system': 'Triclinic', 'symbol': 'P-1', 'number': 2, 'point_group': '-1', 'symprec': 0.1, 'version': '2.0.2'}, 'material_id': 'mp-753395', 'total_magnetization_normalized_formula_units': 5.00000015}]Action:
```
{
  "action": "Final Answer",
  "action_input": "The stable FePO4 has the material id 'mp-753395', belongs to the Triclinic crystal system with space group symbol 'P-1' and number 2. The total magnetization per formula unit is approximately 5.00000015 Bohr magnetons."
}
```Action:
```
{
  "action": "Final Answer",
  "action_input": "The stable FePO4 has the material id 'mp-753395', belongs to the Triclinic crystal system with space group symbol 'P-1' and number 2. The total magnetization per formula unit is approximately 5.00000015 Bohr magnetons."
}
```

> Finished chain.

Observation: {'input': 'Give me the summary of stable FePO4 including material id, space group, and magnetization per formula unit.', 'output': "The stable Fe

{'input': 'What is the magnetic ordering of stable FePO4? Give me its material id, space group, and magnetization per formula unit.',
 'chat_history': [],
 'output': "The stable FePO4 has a ferromagnetic (FM) magnetic ordering. Its material id is 'mp-753395', and it belongs to the Triclinic crystal system with space group symbol 'P-1' and number 2. The total magnetization per formula unit is approximately 5.00000015 Bohr magnetons."}

In [4]:

llm_gpt = ChatOpenAI(
    temperature=0.7,
    model="gpt-3.5-turbo-1106",
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    # streaming=True
)

llm_gpt.invoke(agent_executor.memory.chat_memory.messages[0].content)


AIMessage(content='The magnetic ordering of stable FePO4 is antiferromagnetic. Its material id in the Materials Project database is mp-2133, and its space group is Pnma. The magnetization per formula unit is 4.04 μB.', response_metadata={'finish_reason': 'stop', 'logprobs': None})